In [2]:
import json
import pandas as pd
import os
import numpy as np
import requests
from dotenv import load_dotenv
import pathlib as path

In [3]:
load_dotenv()
api_key = os.getenv("CENSUS_API_KEY")

In [11]:
national_poverty_url = f"https://api.census.gov/data/2022/acs/acs1?get=NAME,B17001_001E,B17001_002E&for=us:1&key={api_key}"
natpov_query = requests.get(national_poverty_url).json()
json.dumps(natpov_query, indent=4)

'[\n    [\n        "NAME",\n        "B17001_001E",\n        "B17001_002E",\n        "us"\n    ],\n    [\n        "United States",\n        "325521470",\n        "40951625",\n        "1"\n    ]\n]'

In [12]:
national_poverty_df = pd.DataFrame(natpov_query)
national_poverty_df

national_poverty_df.columns = national_poverty_df.iloc[0]
national_poverty_df = national_poverty_df[1:]
national_poverty_df.reset_index(drop=True, inplace=True)

national_poverty_df

national_poverty2_df = national_poverty_df[["NAME", "B17001_001E", "B17001_002E"]]
#national_poverty2_df.astype({"Total Population": int, "Est. Pop Below Povert Level": int})

national_poverty2_df

national_poverty2022_df = national_poverty2_df.rename(columns={'NAME': 'County', 'B17001_001E': 'Total Population', 'B17001_002E': 'Est. Pop Below Poverty Level'})

national_poverty2022_df = national_poverty2022_df.astype({"Total Population": int, "Est. Pop Below Poverty Level": int})
national_poverty2022_df["2022 Poverty Rate"] = (national_poverty2022_df["Est. Pop Below Poverty Level"] / national_poverty2022_df["Total Population"])*100

In [6]:
poverty_acs_url = f"https://api.census.gov/data/2022/acs/acs1?get=NAME,B17001_001E,B17001_002E&for=county:*&in=state:*&key={api_key}"
query = requests.get(poverty_acs_url).json()
json.dumps(query, indent=4)

'[\n    [\n        "NAME",\n        "B17001_001E",\n        "B17001_002E",\n        "state",\n        "county"\n    ],\n    [\n        "Baldwin County, Alabama",\n        "241525",\n        "32313",\n        "01",\n        "003"\n    ],\n    [\n        "Calhoun County, Alabama",\n        "111451",\n        "20450",\n        "01",\n        "015"\n    ],\n    [\n        "Cullman County, Alabama",\n        "89595",\n        "16019",\n        "01",\n        "043"\n    ],\n    [\n        "DeKalb County, Alabama",\n        "70267",\n        "13967",\n        "01",\n        "049"\n    ],\n    [\n        "Elmore County, Alabama",\n        "85226",\n        "9133",\n        "01",\n        "051"\n    ],\n    [\n        "Etowah County, Alabama",\n        "101261",\n        "17774",\n        "01",\n        "055"\n    ],\n    [\n        "Houston County, Alabama",\n        "107230",\n        "18773",\n        "01",\n        "069"\n    ],\n    [\n        "Jefferson County, Alabama",\n        "646605"

In [13]:
poverty_acs_df = pd.DataFrame(query)
poverty_acs_df.sort_values(0, ascending=False)

poverty_acs_df.columns = poverty_acs_df.iloc[0]
poverty_acs_df = poverty_acs_df[1:]
poverty_acs_df.reset_index(drop=True, inplace=True)

reorg_poverty_acs_df = poverty_acs_df[['NAME', 'B17001_001E', 'B17001_002E']]
reorg_poverty_acs_df

renamed_poverty_acs_df = reorg_poverty_acs_df.rename(columns={'NAME': 'County', 'B17001_001E': 'Total Population',
                                                                'B17001_002E': 'Est. Pop Below Poverty Level',
                                                                })

renamed_poverty_acs_df['Total Population']=renamed_poverty_acs_df['Total Population'].astype(int)
renamed_poverty_acs_df['Est. Pop Below Poverty Level']=renamed_poverty_acs_df['Est. Pop Below Poverty Level'].astype(int)
renamed_poverty_acs_df['2022 Poverty Rate'] = (renamed_poverty_acs_df['Est. Pop Below Poverty Level'] / renamed_poverty_acs_df['Total Population'])*100

In [8]:
#Merged National & County DFs:

combined_poverty2022_df = pd.concat([national_poverty2022_df, renamed_poverty_acs_df], axis=0)
combined_poverty2022_df

,County,Total Population,Est. Pop Below Poverty Level,2022 Poverty Rate
0,United States,325521470,40951625,12.580315
0,"Baldwin County, Alabama",241525,32313,13.378739
1,"Calhoun County, Alabama",111451,20450,18.348871
2,"Cullman County, Alabama",89595,16019,17.879346
3,"DeKalb County, Alabama",70267,13967,19.877040
...,...,...,...,...
843,"Ponce Municipio, Puerto Rico",128156,68118,53.152408
844,"San Juan Municipio, Puerto Rico",330936,126728,38.293809
845,"Toa Alta Municipio, Puerto Rico",65885,18491,28.065569
846,"Toa Baja Municipio, Puerto Rico",72613,24484,33.718480
